## **1. 모델 이해하기**

합성곱 신경망은 출처에 따라서 합성곱 층을 부르는 단위가 다릅니다.

- 첫번째 표기 방법
합성곱(nn.Cov2d) + 활성화 함수(nn.ReLU)를 하나의 합성곱 층으로 보고, 맥스풀링(nn.MaxPoold2d)은 풀링 층으로 별도 명명합니다.

- 두번째 표기 방법
합성곱(nn.Cov2d) + 활성화 함수(nn.ReLU) + 맥스풀링(nn.MaxPoold2d)을 하나의 합성곱 층으로 봅니다.

다시 말해 풀링도 하나의 층으로 보느냐, 안 보느냐의 차이인데 어느 것이 맞고 틀리다의 문제는 아닙니다. 여기서는 편의를 위해 맥스풀링까지 포함해서 하나의 합성곱 층으로 판단하고 정리하겠습니다.

모델의 아키텍처는 총 3개의 층으로 구성됩니다.

> 1번 레이어: 합성곱층(Convolutional layer)  
> 합성곱(in_channel = 1, out_channel = 32, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU + 맥스 풀링(kernel_size=2, stride=2)
>
> 2번 레이어 : 합성곱층(Convolutional layer)  
>합성곱(in_channel = 32, out_channel = 64, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
>맥스풀링(kernel_size=2, stride=2))
>
> 3번 레이어 : 전결합층(Fully-Connected layer)  
>특성맵을 펼친다. # batch_size × 7 × 7 × 64 → batch_size × 3136
>전결합층(뉴런 10개) + 활성화 함수 Softmax

이를 직접 구현해봅시다.

## **2. 모델 구현하기**

### 2.1 필요한 도구 import 및 입력의 정의

In [1]:
import torch
import torch.nn as nn

In [2]:
# 배치 크기 x 채널 x 높이 x 너비의 크기를 가지는 텐서 선언
inputs = torch.Tensor(1, 1, 28, 28) # 1 x 1 x 28 x 28
print("텐서의 크기 : ", inputs.shape)

텐서의 크기 :  torch.Size([1, 1, 28, 28])


### 2.2 합성곱층과 풀링 선언하기

이제 첫번째 합성곱 층을 구현해봅시다. 1채널 짜리를 입력받아서 32채널을 뽑아내는데 커널 사이즈는 3이고 패딩은 1입니다.

In [3]:
conv1 = nn.Conv2d(1, 32, 3, padding=1)
conv1

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

이제 두번째 합성곱 층을 구현해봅시다. 32채널 짜리를 입력받아서 64채널을 뽑아내고 커널 사이즈는 3, 패딩은 1입니다.

In [4]:
conv2 = nn.Conv2d(32, 64, 3, padding=1)
conv2

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

이제 맥스풀링을 구현해봅시다. 정수 하나를 인자로 넣으면 커널 사이즈와 스트라이드가 해당값으로 지정됩니다.

In [5]:
pool = nn.MaxPool2d(2)
pool

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

### 2.3 구현체를 연결하여 모델 만들기

이제 선언한 것들을 연결해보겠습니다. 우선 입력을 첫번째 합성곱층에 통과시킨 뒤, 텐서의 크기를 보겠습니다.

In [6]:
out = conv1(inputs)
out.shape

torch.Size([1, 32, 28, 28])

32채널의 28 x 28 텐서가 되었씁니다. conv1의 out_channel을 32로 지정했기에 32채널이 나왔고 padding을 1로 설정하고 kerner의 크기를 3 x 3으로 했기에 크기는 유지되었습니다. 이제 맥스풀링을 통과한 후의 텐서 크기를 보겠습니다.

In [7]:
out = pool(out)
out.shape

torch.Size([1, 32, 14, 14])

32채널의 14 x 14의 텐서가 되었습니다. 이제 두번째 합성곱층에 통과시키겠습니다.

In [8]:
out = conv2(out)
out.shape

torch.Size([1, 64, 14, 14])

64채널의 14 x 14의 텐서가 되었습니다. 이 역시 패딩을 1로 하고 3 x 3 커널을 사용했기에 크기는 보존합니다. 이제 이를 맥스풀링을 통과시키겠습니다.

In [9]:
out = pool(out)
out.shape

torch.Size([1, 64, 7, 7])

이제 이 텐서를 펼치는 작업을 할 것입니다. 그런데 펼치기에 앞서 텐서의 n번째 차원을 접근하게 해주는 .size(n)을 먼저 보겠습니다. 현재 out의 크기는 1 x 64 x 7 x 7입니다. 

In [10]:
out.size(0), out.size(1), out.size(2), out.size(3)

(1, 64, 7, 7)

위 결과를 보면 size에 순서대로 저장된 것을 볼 수 있습니다. 이제 이를 가지고 .view()를 사용하여 텐서를 펼치는 작업을 해보겠습니다.

In [11]:
# 첫번째 차원인 배치 차원은 그대로 두고 나머지는 펼침
out = out.view(out.size(0), -1)
out.shape

torch.Size([1, 3136])

배치 차원을 제외하고 모두 하나의 차원으로 통합된 것을 볼 수 있습니다. 이제 이에 대해서 전결합층(Fully-Conected layer)를 통과시켜보겠습니다. 출력층으로 10개의 뉴런을 배치하여 10개의 차원 텐서로 변환합니다.

In [12]:
fc = nn.Linear(3136, 10) # input_dim = 3136, output_dim = 10
out = fc(out)
out.shape

torch.Size([1, 10])

## **3. CNN으로 MNIST 분류하기**

In [13]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [33]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [34]:
mnist_train = dsets.MNIST(root='MNIST_data/', # 저장 경로 지정
                        train = True, # 훈련 데이터 다운로드
                        transform=transforms.ToTensor(), # 텐서로 변환
                        download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 저장 경로 지정
                        train = False, # 테스트 데이터 다운로드
                        transform=transforms.ToTensor(), # 텐서로 변환
                        download=True)

데이터로더를 사용하여 배치 크기를 지정합니다.

In [35]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, 
                                        batch_size=batch_size,
                                        shuffle=True,
                                        drop_last=True)

In [40]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째 층
        # ImgIn shape = (?, 28, 28, 1)
        # Conv       -> (?, 28, 28, 32)
        # Pool       -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # 두번째 층
        # ImgIn shape = (?, 14, 14, 32)
        # Conv       -> (?, 14, 14, 64)
        # Pool       -> (?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # 전결합층 7 x 7 x 64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
        
        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)
    
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1) # 전결합층을 위해서 flatten
        out = self.fc(out)
        return out

모델을 정의합시다.

In [41]:
# CNN 모델 정의
model = CNN().to(device)

# 비용 함수와 옵티마이저를 정의합니다.
criterion = torch.nn.CrossEntropyLoss().to(device) # 비용 함수에 소프트맥스 함수 포함
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [42]:
total_batch = len(data_loader)
print("총 배치의 수:", total_batch)

총 배치의 수: 600


총 배치의 수는 600입니다. 그런데 배치 크기를 100으로 설정했으므로 결국 훈련 데이터는 총 60,000개라는 의미입니다. 이제 모델을 훈련시켜보겠습니다. (시간이 꽤 오래걸립니다.)

In [43]:
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니배치, Y는 레이블
        # image is already size of (28 x 28), no reshape
        # label is not one-hoe endcoded
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.205349803
[Epoch:    2] cost = 0.0577153377
[Epoch:    3] cost = 0.0415475257
[Epoch:    4] cost = 0.0329876952
[Epoch:    5] cost = 0.0268548336
[Epoch:    6] cost = 0.0224587414
[Epoch:    7] cost = 0.0187493172
[Epoch:    8] cost = 0.0164931137
[Epoch:    9] cost = 0.0134951435
[Epoch:   10] cost = 0.010925849
[Epoch:   11] cost = 0.00908607058
[Epoch:   12] cost = 0.00838652812
[Epoch:   13] cost = 0.00773216691
[Epoch:   14] cost = 0.0066355383
[Epoch:   15] cost = 0.00483957073


이제 테스트를 해보겠습니다.

In [44]:
# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print("Accuracy:", accuracy.item())

C:\Users\delphinus\anaconda3\envs\nlp\lib\site-packages\torchvision\datasets\mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\delphinus\anaconda3\envs\nlp\lib\site-packages\torchvision\datasets\mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9869999885559082


다음 챕터에선 층을 더 쌓아보겠습니다